In [2]:

from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
#InMemorySaver is a checkpointer that saves the state of the graph in memory. It is useful for testing and debugging.
#It is used to implement persistence in the graph, which allows us to save the state of the graph and load it later.
#Saves state in RAM
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [5]:

def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:

graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [8]:
#Data is stored against the thread_id in the checkpointer. We can use the same thread_id to retrieve the state of the graph later.
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza get a job as a motivational speaker?\n\nBecause it always knew how to **slice** through problems!',
 'explanation': 'This joke is a **pun**, playing on the double meaning of the word "slice."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Pizza is something that is literally **sliced** into pieces before it\'s eaten. It\'s a fundamental characteristic of pizza.\n\n2.  **Figurative Meaning (Problem Solving):** The phrase "to **slice through problems**" is an idiom. It means to effectively and decisively deal with difficulties, to cut through complexity, or to overcome obstacles with ease.\n\nThe humor comes from the joke attributing the *literal* action of pizza (being sliced) to the *figurative* ability of a good motivational speaker (helping people "slice through" their problems). It\'s a silly, unexpected connection that makes you groan and chuckle. The pizza is "good at its job" because its very nature involves "slicing."

In [9]:
#To know the current state of the graph, we can use the get_state method of the workflow. It will return the current state of the graph along with the data stored in the checkpointer.
workflow.get_state(config=config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job as a motivational speaker?\n\nBecause it always knew how to **slice** through problems!', 'explanation': 'This joke is a **pun**, playing on the double meaning of the word "slice."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Pizza is something that is literally **sliced** into pieces before it\'s eaten. It\'s a fundamental characteristic of pizza.\n\n2.  **Figurative Meaning (Problem Solving):** The phrase "to **slice through problems**" is an idiom. It means to effectively and decisively deal with difficulties, to cut through complexity, or to overcome obstacles with ease.\n\nThe humor comes from the joke attributing the *literal* action of pizza (being sliced) to the *figurative* ability of a good motivational speaker (helping people "slice through" their problems). It\'s a silly, unexpected connection that makes you groan and chuckle. The pizza is "good at its job" because its very nature 

In [10]:
#To get intermediate state value of thread_id "1", we can use the get_state_history method of the workflow. It will return the history of states of the graph along with the data stored in the checkpointer.
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job as a motivational speaker?\n\nBecause it always knew how to **slice** through problems!', 'explanation': 'This joke is a **pun**, playing on the double meaning of the word "slice."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Pizza is something that is literally **sliced** into pieces before it\'s eaten. It\'s a fundamental characteristic of pizza.\n\n2.  **Figurative Meaning (Problem Solving):** The phrase "to **slice through problems**" is an idiom. It means to effectively and decisively deal with difficulties, to cut through complexity, or to overcome obstacles with ease.\n\nThe humor comes from the joke attributing the *literal* action of pizza (being sliced) to the *figurative* ability of a good motivational speaker (helping people "slice through" their problems). It\'s a silly, unexpected connection that makes you groan and chuckle. The pizza is "good at its job" because its very nature

In [11]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'batman'}, config=config2)

{'topic': 'batman',
 'joke': "What's Batman's favorite part of a joke?\n\nThe punchline!",
 'explanation': 'This joke plays on the **double meaning of the word "punchline."**\n\nHere\'s the breakdown:\n\n1.  **Standard Meaning of "Punchline":** In the context of a joke, the "punchline" is the final phrase or sentence that delivers the humor, the unexpected twist, or the funny conclusion. It\'s the part that makes you laugh.\n\n2.  **The Batman Connection:** Batman is a superhero famous for fighting crime. His primary method of doing so often involves physical combat, where he literally **"punches"** villains.\n\nThe joke makes a pun by suggesting that Batman\'s "favorite part" isn\'t just the humorous ending of a story, but the literal act of delivering a **"punch"** (as in a fistfight) – something he does very well and very often in his crime-fighting career.\n\nThe humor comes from the unexpected twist of applying a word from the world of comedy ("punchline") to a literal action from

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job as a motivational speaker?\n\nBecause it always knew how to **slice** through problems!', 'explanation': 'This joke is a **pun**, playing on the double meaning of the word "slice."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Pizza is something that is literally **sliced** into pieces before it\'s eaten. It\'s a fundamental characteristic of pizza.\n\n2.  **Figurative Meaning (Problem Solving):** The phrase "to **slice through problems**" is an idiom. It means to effectively and decisively deal with difficulties, to cut through complexity, or to overcome obstacles with ease.\n\nThe humor comes from the joke attributing the *literal* action of pizza (being sliced) to the *figurative* ability of a good motivational speaker (helping people "slice through" their problems). It\'s a silly, unexpected connection that makes you groan and chuckle. The pizza is "good at its job" because its very nature 

*FAULT TOLERANCE*

In [13]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [20]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3 : str

In [21]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"step3": "done"}

In [22]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


*TIME TRAVEL*

In [15]:
#TIME TRAVEL : We can now retrieve the state of the graph at the point of interruption and resume execution from there.
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [16]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

EmptyInputError: Received no input for __start__